First part of the chain, just setup api key (on `azure` in this example). This cell prints the name of the base deployment just to verify that we correctly loaded the .env file.

Remember you need a .env file with the following content:

```
OPENAI_API_KEY=xxxxxxxxx
OPENAI_API_BASE=https://alkopenai2.openai.azure.com/
HUGGINGFACEHUB_API_TOKEN=xxxxxxx
PINECONE_KEY=xxxxxxx
PINECONE_ENV=us-west1-gcp-free
SERPAPI_API_KEY=xxxxx
```

In [ ]:
import os
import openai
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# print base to verify you are pointint to the right deployment
print(openai.api_base)


Now to work langchain nees to work with LLM, you start creating some LLM in this situation we use OpenAI in azure in different flavors.

The important aspect is that some of the llm models are `chat models` and others are `text models`. The chat models are trained to answer questions and the text models are trained to generate text. You will use both of these in **different situation**.

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI  

# Pay attention that gpt35 and gpt4 are chat based llms
gpt35 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt35", 
    model_name="gpt-35-turbo"
)

gpt4 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt4", 
    model_name="gpt-4"
)

# this is not a Chat model this is a text model
davinci = AzureOpenAI(
	temperature=0,
	openai_api_version="2023-03-15-preview",
    deployment_name="text-davinci-003", 
	model_name="text-davinci-003"
)

Langchain first function: ability to create a `prompt` with placeholders to better manage the prompt. The key here is simply the ability to create a string with placeholders.

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

# pay attention we are not using python f" style of strings.
prompt_template_string = """Translate sentence surrounded by triple ticks to {target_lang}.

```{sentence}```
"""

# Create a prompt object from the template 
# super important THE MODEL IS OF TYPE CHATPROMPTTEMPLATE
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template_string)

# now you have a complex template object that as an example has input variables
pprint(chat_prompt_template.messages[0].input_variables)

# now you can generate a real prompt substituing input variables.
chat_sample_prompt = chat_prompt_template.format_messages(target_lang="de", sentence="This is a simple experiment to test langchain")
pprint(f"Type of the chat_sample_prompt is a list of {len(chat_sample_prompt)} first element type is  {type(chat_sample_prompt[0])}")

# you can create a simple template
prompt_template = PromptTemplate.from_template(prompt_template_string)

pprint(f"Type of the prompt_template is {type(prompt_template)} and input variables are {prompt_template.input_variables}")
sample_prompt = prompt_template.format(target_lang="de", sentence="This is a simple experiment to test langchain")

pprint(f"Type of the sample_prompt is {type(sample_prompt)}")
pprint(sample_prompt   )

In [ ]:
# this will show that the chat sample prompt is a list of objects and each object has a type
# and a content attribute that contains the text of the prompt
first_element = chat_sample_prompt[0]
pprint(f"Type of first element: {type(first_element)}")
print(first_element.content)

`Chat Models` can accept the textPrompt object, that is basically an array, and simply return another piece of the conversation. We are actually creating a `prompt` with a single message and the chat model will answer with another piece of object.

You will see that the textPrompt is a `list` containing HumanMessages or AIMessages. This is the standard way to dialogate with a chat model.

In [ ]:
# now you can call an api through the wrapper
print(f"textPrompt is of type {type(chat_sample_prompt)} and first element is of type {type(chat_sample_prompt[0])}")  
result = gpt35(chat_sample_prompt)  
print(f"Result is of type {type(result)}")
pprint(result)  

#actually it is more useful to print the content
pprint(f"Content of the answer is = {result.content}")

Davinci is a simple completion model, so it will simply complete my prompt. 
1. I need to pass the prompt as string not the entire prompt (only chain models supports    the entire prompt)
2. I will simply return text

In [ ]:
# if you have a ChatPromptTemplate you need to pass a single string content.
result = davinci(chat_sample_prompt[0].content)
print(result)

# it is more standard to have a standard PromptTemplate that uses .format to create
# a sample string
result = davinci(sample_prompt)
pprint(result)

Basically `ChatPromptTemplate` and `PromptTemplate` are used respectively with models of type Chat or Text. Please always remember this distinction between the two.